In [141]:
import re
from datetime import datetime, timedelta
import numpy as np
from collections import defaultdict

In [82]:
def read_input(filename):
    expr = re.compile(r'\[(.*)\] (.*)')
    sched = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            m = expr.match(line.strip())
            sched.append([datetime.fromisoformat(m[1]), m[2]])
    return sched

In [125]:
sched = read_input('04_input.txt')

In [126]:
sched = sorted(sched, key=lambda x: x[0])

In [135]:
asleep = {}
i = 0
while i < len(sched):
    dt_date = sched[i][0]
    if dt_date.hour == 23:
        dt_date += timedelta(hours=1)
    month = dt_date.month
    day = dt_date.day
    date = (month, day)
    guard = int(sched[i][1].split(' ')[1][1:])
    i += 1
    if i >= len(sched):
        break
    s = np.zeros(60)
    while not sched[i][1].startswith('Guard '):
        minute = sched[i][0].minute
        if sched[i][1].startswith('falls'):
            s[minute:] = 1
        elif sched[i][1].startswith('wakes'):
            s[minute:] = 0
        i += 1
        if i >= len(sched):
            break
    if (date, guard) in asleep:
        print(f'{(date, guard)} already exists!')
    asleep[date, guard] = s

In [136]:
minutes_asleep = {}
for (date, guard), s in asleep.items():
    minutes_asleep[guard] = minutes_asleep.get(guard, 0) + np.sum(s)

In [137]:
sleepy_guard = max(minutes_asleep.items(), key=lambda x: x[1])[0]

In [138]:
sleeps = []
for (date, guard), s in asleep.items():
    if guard == sleepy_guard:
        sleeps.append(s)

In [139]:
sleepy_minute = np.argmax(np.sum(np.array(sleeps), axis=0))

In [140]:
sleepy_guard * sleepy_minute

115167

In [147]:
guard_sleeps = defaultdict(list)
for (date, guard), s in asleep.items():
    guard_sleeps[guard].append(s)
for guard in guard_sleeps:
    guard_sleeps[guard] = np.array(guard_sleeps[guard])

In [157]:
tot_max_sleep = 0
tot_max_minute = 0
sleepies_guard = 0
for guard, s in guard_sleeps.items():
    spm = np.sum(s, axis=0)
    max_sleep = np.max(spm)
    max_minute = np.argmax(spm)
    if max_sleep > tot_max_sleep:
        tot_max_minute = max_minute
        tot_max_sleep = max_sleep
        sleepiest_guard = guard

In [160]:
sleepiest_guard * tot_max_minute

32070